In [1]:
import xml.etree.ElementTree as ET
import urllib.request, urllib.parse, urllib.error
from tqdm import tqdm_notebook
from datetime import date
from collections import namedtuple
from collections import Counter
import re
from time import sleep

In [2]:
Proposition = namedtuple('Proposition', ['data_votacao', 'cod', 'tipo', 'numero', 'ano_proposicao'])
def getPropositionsInYear(year):
    props = []
    url = 'http://www.camara.gov.br/SitCamaraWS/Proposicoes.asmx/ListarProposicoesVotadasEmPlenario?ano=' + str(year) + '&tipo='
    document = urllib.request.urlopen (url).read()
    root = ET.fromstring(document)
    for prop in root.findall('proposicao'):
        cod = prop.find('codProposicao').text
        nome = prop.find('nomeProposicao').text
        content = nome.split()
        tipo = content[0]
        content = content[1].split('/')
        numero = content[0]
        ano_proposicao = content[1]
        data_votacao = prop.find('dataVotacao').text
        h = cod + ano_proposicao
        props.append(Proposition(data_votacao, cod, tipo, numero, ano_proposicao))
    return props
def get_votos(prop): 
    url = 'http://www.camara.gov.br/SitCamaraWS/Proposicoes.asmx/ObterVotacaoProposicao?tipo=' + prop.tipo + '&numero=' + prop.numero + '&ano=' + prop.ano_proposicao
    tries = 0
    while tries < 10:
        try:
            votos = []
            document = urllib.request.urlopen(url).read()
            root = ET.fromstring(document)
            for votacoes in root.findall('Votacoes'):
                for votacao in votacoes.findall('Votacao'):
                    hora = votacao.get('Hora').strip()
                    data = votacao.get('Data').strip()
                    codSessao = votacao.get('codSessao').strip()
                    for vote in votacao.find('votos'):
                        voto = vote.get('Voto').strip()
                        if voto == '-':
                            continue
                        item = []
                        item.append(prop.cod)
                        item.append(prop.numero)
                        item.append(prop.tipo)
                        item.append(codSessao)
                        item.append(hora + ' ' + data)
                        item.append(vote.get('Nome').strip())
                        item.append(vote.get('Partido').strip())
                        item.append(vote.get('UF').strip())
                        item.append(voto)
                        votos.append(item)
            break
        except Exception as e:
            tries += 1
            if tries == 10:
                print(tries, url, e)
    #sleep(1)
    return votos

In [3]:
props = []
for ano in tqdm_notebook(range(1991, date.today().year+1), position=1, desc='Carregando', ncols=700):
    props.extend(getPropositionsInYear(ano))

In [4]:
len(props), props[:5]

(3624,
 [Proposition(data_votacao='17/12/1991', cod='20976', tipo='PL', numero='4580', ano_proposicao='1990'),
  Proposition(data_votacao='07/11/1991', cod='167035', tipo='PDC', numero='63', ano_proposicao='1991'),
  Proposition(data_votacao='28/06/1991', cod='167073', tipo='PDC', numero='66', ano_proposicao='1991'),
  Proposition(data_votacao='19/09/1991', cod='167073', tipo='PDC', numero='66', ano_proposicao='1991'),
  Proposition(data_votacao='04/09/1991', cod='167304', tipo='PDC', numero='87', ano_proposicao='1991')])

In [5]:
from multiprocessing import cpu_count
from multiprocessing import Pool

In [6]:
with Pool(processes=cpu_count()) as p:
    all_votes = list(p.imap_unordered(get_votos, tqdm_notebook(props, total=len(props))))

10 http://www.camara.gov.br/SitCamaraWS/Proposicoes.asmx/ObterVotacaoProposicao?tipo=REQ&numero=12&ano=2012 HTTP Error 400: Bad Request
10 http://www.camara.gov.br/SitCamaraWS/Proposicoes.asmx/ObterVotacaoProposicao?tipo=REQ&numero=12&ano=2012 HTTP Error 400: Bad Request
10 http://www.camara.gov.br/SitCamaraWS/Proposicoes.asmx/ObterVotacaoProposicao?tipo=REQ&numero=12&ano=2012 HTTP Error 400: Bad Request
10 http://www.camara.gov.br/SitCamaraWS/Proposicoes.asmx/ObterVotacaoProposicao?tipo=REQ&numero=12&ano=2012 HTTP Error 400: Bad Request
10 http://www.camara.gov.br/SitCamaraWS/Proposicoes.asmx/ObterVotacaoProposicao?tipo=REQ&numero=12&ano=2012 HTTP Error 400: Bad Request
10 http://www.camara.gov.br/SitCamaraWS/Proposicoes.asmx/ObterVotacaoProposicao?tipo=REQ&numero=12&ano=2012 HTTP Error 400: Bad Request
10 http://www.camara.gov.br/SitCamaraWS/Proposicoes.asmx/ObterVotacaoProposicao?tipo=REQ&numero=12&ano=2012 HTTP Error 400: Bad Request
10 http://www.camara.gov.br/SitCamaraWS/Proposic

10 http://www.camara.gov.br/SitCamaraWS/Proposicoes.asmx/ObterVotacaoProposicao?tipo=REQ&numero=1118&ano=2015 HTTP Error 400: Bad Request
10 http://www.camara.gov.br/SitCamaraWS/Proposicoes.asmx/ObterVotacaoProposicao?tipo=REQ&numero=1121&ano=2015 HTTP Error 400: Bad Request
10 http://www.camara.gov.br/SitCamaraWS/Proposicoes.asmx/ObterVotacaoProposicao?tipo=REQ&numero=1125&ano=2015 HTTP Error 400: Bad Request
10 http://www.camara.gov.br/SitCamaraWS/Proposicoes.asmx/ObterVotacaoProposicao?tipo=REQ&numero=1129&ano=2015 HTTP Error 400: Bad Request
10 http://www.camara.gov.br/SitCamaraWS/Proposicoes.asmx/ObterVotacaoProposicao?tipo=REQ&numero=1126&ano=2015 HTTP Error 400: Bad Request

10 http://www.camara.gov.br/SitCamaraWS/Proposicoes.asmx/ObterVotacaoProposicao?tipo=REQ&numero=7&ano=2017 HTTP Error 400: Bad Request
10 http://www.camara.gov.br/SitCamaraWS/Proposicoes.asmx/ObterVotacaoProposicao?tipo=REQ&numero=7&ano=2017 HTTP Error 400: Bad Request
10 http://www.camara.gov.br/SitCamaraWS

In [7]:
# ATUALMENTE EXISTEM 16992733
sum([len(vote) for vote in all_votes])

16992733

In [14]:
all_votes_2 = []
for prop_votes in tqdm_notebook(all_votes, total=len(all_votes)):
    all_votes_2.extend(prop_votes)
len(all_votes_2)

16992733

In [15]:
fn = 'data/votos.csv.gz'
columns = ['cod_prop', 'num_prop', 'tipo', 'codSessao', 'data_hora', 'nome_dep', 'partido', 'uf', 'voto']

In [16]:
len(all_votes_2)

16992733

In [17]:
import pandas as pd
df = pd.DataFrame(all_votes_2, columns=columns)
df.head()

,cod_prop,num_prop,tipo,codSessao,data_hora,nome_dep,partido,uf,voto
0,167035,63,PDC,2081,17:05 7/11/1991,Alceste Almeida,PTB,RR,Não
1,167035,63,PDC,2081,17:05 7/11/1991,Júlio Cabral,PTR,RR,Abstenção
2,167035,63,PDC,2081,17:05 7/11/1991,João Fagundes,PMDB,RR,Sim
3,167035,63,PDC,2081,17:05 7/11/1991,Marcelo Luz,PTR,RR,Sim
4,167035,63,PDC,2081,17:05 7/11/1991,Gilvan Borges,PMDB,AP,Não


In [18]:
df.to_csv(fn, compression='gzip')

In [11]:
import gzip
with gzip.open(fn, 'wt') as fout:
    fout.write(','.join(columns))
    for prop_votes in tqdm_notebook(all_votes, total=len(all_votes)):
        for row in prop_votes:
            fout.write(','.join(row))

In [ ]:
df2 = df[df['voto'].str.contains("Sim|Não|Branco|Obstrução")]

In [ ]:
df.shape[0] - df2.shape[0]

In [ ]:
all_votes_2 = list(map(get_votos, tqdm_notebook(props, total=len(props))))

In [ ]:
df.head()

In [ ]:
def get_votes(proposition):
    ori,objs_votacao = get_orientacoes(proposition)
    items = []
    for obj_voto in objs_votacao:
        for vote in obj_voto.find('votos'):
            item = []
            item.append(proposition.cod)
            item.append(proposition.numero)
            item.append(obj_voto.get('codSessao'))
            item.append(obj_voto.get('Data') + ' ' + obj_voto.get('Hora'))
            item.append(vote.get('Nome'))
            item.append(vote.get('Partido').strip())
            item.append(vote.get('UF'))
            item.append(vote.get('Voto'))
            items.append(item)
    return items

In [ ]:
def flat_map(f, seq):
    return sum(map(f,seq), [])

In [ ]:
aaa = flat_map(get_votes, tqdm_notebook(props, total=len(props)))

In [ ]:
aaa

In [ ]:
df

In [ ]:
all_votes = []
for prop in tqdm_notebook(props, total=len(props)):
    ori,objs_votacao = get_orientacoes(prop)
    for obj_voto in objs_votacao:
        for vote in obj_voto.find('votos'):
            item = []
            item.append(prop.cod)
            item.append(prop.numero)
            item.append(obj_voto.get('codSessao'))
            item.append(obj_voto.get('Data'))
            item.append(obj_voto.get('Hora'))
            item.append(vote.get('Hora'))
            item.append(vote.get('Partido'))
            item.append(vote.get('UF'))
            item.append(vote.get('Voto'))
            all_votes.append(item)

In [ ]:
objs_votacao[0].find('votos')[2].keys()

In [ ]:
objs_votacao[0].find('votos')[2].get('Partido')

In [ ]:
objs_votacao[0].keys()

In [ ]:
id_s = 0
for obj_voto in objs_votacao:
    print(obj_voto.get('codSessao'), obj_voto.get('Data'), obj_voto.get('Hora'), obj_voto.get('Resumo'))

In [ ]:
xmlstr = ET.tostring(objs_votacao[1], encoding='utf8', method='xml')
print(str(xmlstr))

In [ ]:
reversed_props = list(reversed(props))
orientacoes = []
for prop in tqdm_notebook(reversed_props, position=1, desc='Carregando', ncols=700):
    orientacoes.extend(get_orientacoes(prop))

In [ ]:
print(orientacoes[0])
with open('data/orientacoes.csv', 'w') as fout:
    for orientacao in orientacoes:
        fout.write('%s\t%s\t%s\t%s\t%s\n' % (orientacao.data_votacao,
                                             orientacao.cod_prop,
                                             orientacao.cod_sessao,
                                             orientacao.sigla,
                                             orientacao.orientacao)) 

In [ ]:
def filter_bancada(orientacoes):
    orientacoes_filtered = []
    for orientacao in tqdm_notebook(orientacoes, desc="Initial Filter"):
        if '/' in orientacao.sigla:
            for sigla in orientacao.sigla.split('/'):
                orientacoes_filtered.append(
                    Orientacao(orientacao.data_votacao,
                               orientacao.cod_prop,
                               orientacao.cod_sessao,
                               sigla,
                               orientacao.orientacao))
        elif 'Repr.' in orientacao.sigla:
            orientacoes_filtered.append(
                Orientacao(orientacao.data_votacao,
                           orientacao.cod_prop,
                           orientacao.cod_sessao,
                           orientacao.sigla.replace('Repr.',''),
                           orientacao.orientacao)
            )
        else:
            orientacoes_filtered.append(orientacao)
    orientacoes_filtered2 = []
    parties = ['Pmdb', 'Psd', 'Pt', 'Dem', 'Pen', 'Pan', 'Pv', 'Sd', 'Pros', 'Pps', 'Psdc',
               'Psdb', 'PTdoB', 'Ptb', 'Psl', 'Ptc', 'Ptn', 'Pp', 'Psb', 'Pdt',
               'Pcdob', 'Psc', 'Pode', 'Podemos', 'PCdoB', 'Avante', 'Phs', 'PtdoB', 'Ptdob',
               'Prb', 'Prtb', 'Prp', 'Pr', 'Pmn']
    for orientacao in tqdm_notebook(orientacoes_filtered, desc="Split bancada"):
        for party in sorted(parties, key=lambda x: len(x), reverse=True):
            if party in orientacao.sigla:
                orientacoes_filtered2.append(
                    Orientacao(orientacao.data_votacao,
                               orientacao.cod_prop,
                               orientacao.cod_sessao,
                               party.upper(),
                               orientacao.orientacao))
                orientacao = Orientacao(orientacao.data_votacao,
                       orientacao.cod_prop,
                       orientacao.cod_sessao,
                       orientacao.sigla.replace(party,'_'),
                       orientacao.orientacao)
        orientacoes_filtered2.append(orientacao)
    orientacoes_filtered = orientacoes_filtered2.copy()
    orientacoes_filtered2 = []
    for orientacao in tqdm_notebook(list(orientacoes_filtered), desc="Apply aliases"):
        if "_" in orientacao.sigla:
            continue
        elif 'Minoria' == orientacao.sigla or 'Maioria' == orientacao.sigla:
            orientacoes_filtered2.append(
                Orientacao(orientacao.data_votacao,
                           orientacao.cod_prop,
                           orientacao.cod_sessao,
                           orientacao.sigla.upper(),
                           orientacao.orientacao)
            )
        elif 'MDB' == orientacao.sigla:
            orientacoes_filtered2.append(
                Orientacao(orientacao.data_votacao,
                           orientacao.cod_prop,
                           orientacao.cod_sessao,
                           'PMDB',
                           orientacao.orientacao)
            )
        elif 'SDD' == orientacao.sigla or 'Solidaried' == orientacao.sigla :
            orientacoes_filtered2.append(
                Orientacao(orientacao.data_votacao,
                           orientacao.cod_prop,
                           orientacao.cod_sessao,
                           'SD',
                           orientacao.orientacao)
            )
        elif 'PODE' == orientacao.sigla:
            orientacoes_filtered2.append(
                Orientacao(orientacao.data_votacao,
                           orientacao.cod_prop,
                           orientacao.cod_sessao,
                           'PODEMOS',
                           orientacao.orientacao)
            )
        else:
            orientacoes_filtered2.append(orientacao)
    return orientacoes_filtered2

In [ ]:
bancadas = set([orientacao.sigla for orientacao in orientacoes])
sorted(list(bancadas), key=lambda x: len(x), reverse=True)

In [ ]:
orientacoes_filtered = filter_bancada(orientacoes)

In [ ]:
bancadas = Counter([orientacao.sigla for orientacao in orientacoes_filtered])
sorted(bancadas.items(), key=lambda x: x[1], reverse=True)

In [ ]:
len(orientacoes_filtered)

In [ ]:
with open('data/orientacoes-filtered.csv', 'w') as fout:
    for orientacao in orientacoes_filtered:
        fout.write('%s\t%s\t%s\t%s\t%s\n' % (orientacao.data_votacao,
                                             orientacao.cod_prop,
                                             orientacao.cod_sessao,
                                             orientacao.sigla,
                                             orientacao.orientacao)) 

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.DataFrame( [ [ ori.data_votacao, ori.cod_prop, ori.sigla, ori.orientacao  ] for ori in orientacoes_filtered ],
                  columns=['data_votacao', 'cod_prop', 'sigla', 'orientacao'] )
parties = list(set(df['sigla']))
le = LabelEncoder()
df['orientacao'] = le.fit_transform(df['orientacao'])
#df['data_votacao'] = df['data_votacao'].apply( lambda x: x.split('/')[-1] )
df = df.pivot_table(index=['data_votacao','cod_prop'], columns=['sigla'], values=['orientacao'], aggfunc='first' )

In [ ]:
df.head()

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.preprocessing import Imputer

X = df['orientacao'].as_matrix().T
imp = Imputer(axis=1)
X = imp.fit_transform(X)

pca = PCA(n_components=2)
X_r = pca.fit(X).transform(X)

In [ ]:
colors = ['gray', 'navy', 'turquoise', 'darkorange', 'red']
parties.index('PT'), parties.index('PSDB'), parties.index('PMDB'), parties.index('PSL') 

def def_col(i, parties):
    if i == parties.index('PT'):
        return 'red'
    if i == parties.index('PSOL'):
        return 'darkred'
    if i == parties.index('PSDB'):
        return 'turquoise'
    if i == parties.index('PMDB'):
        return 'green'
    if i == parties.index('PSL'):
        return 'blue'
    return 'gray'

all_colors = [ def_col(i, parties) for i,col in enumerate(X_r) ]


In [ ]:
plt.scatter(X_r.T[0], X_r.T[1], color=all_colors)

In [ ]:
df = pd.DataFrame( [ [ ori.data_votacao, ori.cod_prop, ori.sigla, ori.orientacao  ] for ori in orientacoes_filtered ],
                  columns=['data_votacao', 'cod_prop', 'sigla', 'orientacao'] )
parties = list(set(df['sigla']))
le = LabelEncoder()
df['orientacao'] = le.fit_transform(df['orientacao'])
df['data_votacao'] = df['data_votacao'].apply( lambda x: x.split('/')[-1] )
df = df.pivot_table(index=['cod_prop'], columns=['sigla','data_votacao'], values=['orientacao'], aggfunc='first' )
df.head()

In [ ]:
df['orientacao'].head().T.head()

In [ ]:
df

In [ ]:
for year, group_year in df.T.groupby('data_votacao'):
    
    group_year = group_year.fillna(value=0)
    group_year = group_year.loc[:, (group_year.sum(axis=0) != 0.)]
    X = group_year.as_matrix()
    imp = Imputer(axis=1)
    X = imp.fit_transform(X)

    pca = PCA(n_components=8 )
    X_r = pca.fit(X).transform(X)
    
    print(group_year)
    

In [ ]:
group_sigla

In [ ]:
X = df['orientacao'].fillna(value=-1).as_matrix().T

pca = PCA(n_components = 50, svd_solver = 'full')
X_r_50 = pca.fit(X).transform(X)

pca = PCA(n_components=2)
X_r_2 = pca.fit(X_r_50).transform(X_r_50)

In [ ]:
df['orientacao']

In [ ]:
def def_col_df(party):
    if party == 'PT':
        return 'red'
    if party == 'PSOL':
        return 'darkred'
    if party == 'PSDB':
        return 'turquoise'
    if party == 'PMDB':
        return 'green'
    if party == 'PSL':
        return 'blue'
    return 'gray'
import numpy as np

In [ ]:
df_X_r_2 = pd.DataFrame(X_r_2)
parties_df, year_df = list(zip(*df['orientacao'].T.index))
df_X_r_2['parties'] = list(parties_df)
df_X_r_2['year'] = list(year_df)

In [ ]:
from collections import Counter
sorted(Counter(df_X_r_2['parties']).items(), key=lambda x: x[1],  reverse=True)

In [ ]:
for year, group in df_X_r_2.groupby('year'):
    #plt.scatter(X_r.T[0], X_r.T[1], color=all_colors)
    cols = [def_col_df(party) for party in list(group['parties'])]
    group.plot.scatter(x=0, y=1, title=year, colors=cols)
    
    #all_colors = [ def_col(i, group[parties]) for i,col in enumerate(group[parties]) ]

In [ ]:
df_X_r_2

In [ ]:
df3['data_votacao']